<a href="https://colab.research.google.com/github/m-mazurkiewicz/AnalysisOfUnstructuredData/blob/master/ResNet34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
from keras.models import Sequential, load_model, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D as Conv2D, MaxPooling2D, AveragePooling2D, Input, ZeroPadding2D, Add
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
import keras

from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()

import sys
from keras.preprocessing.image import ImageDataGenerator, array_to_img, save_img
from matplotlib import pyplot as plt
import skimage.io as io
import numpy as np
import os
from tqdm import tqdm
from google.colab import drive

Wait for 8 seconds...
TensorBoard link:
https://5f4dfdfb.ngrok.io


In [11]:
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [0]:
num_of_classes = 4
base_dir_processed = '/content/gdrive/My Drive/PWr_AlexNet_data/processed/'
data_set = 'no-padding/resize/'

In [0]:
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.1

In [0]:
image_gen = ImageDataGenerator(
    featurewise_center=True,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=(0.5,1.5),
    shear_range=0.01,
    zoom_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True,
    validation_split=VALIDATION_SPLIT
)

In [61]:
number_of_images_for_fit = -1

all_images = []
for class_name in os.listdir(base_dir_processed + data_set + 'train'):
  for image_path in tqdm(os.listdir(base_dir_processed + data_set + 'train/' + class_name)[:number_of_images_for_fit]):
    img = io.imread(base_dir_processed + data_set + 'train/' + class_name + '/' + image_path)
    all_images.append(img)
x_train = np.array(all_images)

image_gen.fit(x_train)

generator = image_gen.flow_from_directory(
    base_dir_processed + data_set + '/' + 'train',
        target_size=(224,224),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training')

validation_generator = image_gen.flow_from_directory(
    base_dir_processed + data_set + '/' + 'train',
        target_size=(224,224),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation')

100%|██████████| 1109/1109 [00:02<00:00, 472.17it/s]


Found 4693 images belonging to 4 classes.
Found 520 images belonging to 4 classes.


In [0]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2 = filters

    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2a')(X) #, kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b')(X) #, kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
#     X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
#     X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c')(X) #, kernel_initializer=glorot_uniform(seed=0))(X)
#     X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###

    return X

In [0]:
def convolutional_block(X, f, filters, stage, block, s=2):
    """
    Implementation of the convolutional block as defined in Figure 4

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters=F1, kernel_size=(f, f), strides=(s, s), padding='valid', name=conv_name_base + '2a')(X) #, kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b')(X) #, kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
#     X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
#     X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c')(X) #, kernel_initializer=glorot_uniform(seed=0))(X)
#     X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters=F2, kernel_size=(f, f), strides=(s, s), padding='valid', name=conv_name_base + '1')(X_shortcut) #, kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###
    return X

In [0]:
X_input = Input(shape=(224, 224, 3))

# Zero-Padding
X = ZeroPadding2D((3, 3))(X_input)

# Stage 1
X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(X) #, kernel_initializer=glorot_uniform(seed=0))(X)
X = BatchNormalization(axis=3, name='bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3, 3), strides=(2, 2))(X)

# Stage 2
X = convolutional_block(X, f=3, filters=[64, 64], stage=2, block='a', s=1)
X = identity_block(X, 3, [64, 64], stage=2, block='b')
X = identity_block(X, 3, [64, 64], stage=2, block='c')

### START CODE HERE ###

# Stage 3 (≈4 lines)
X = convolutional_block(X, f=3, filters=[128, 128], stage=3, block='a', s=2)
X = identity_block(X, 3, [128, 128], stage=3, block='b')
X = identity_block(X, 3, [128, 128], stage=3, block='c')
X = identity_block(X, 3, [128, 128], stage=3, block='d')

# Stage 4 (≈6 lines)
X = convolutional_block(X, f=3, filters=[256, 256], stage=4, block='a', s=2)
X = identity_block(X, 3, [256, 256], stage=4, block='b')
X = identity_block(X, 3, [256, 256], stage=4, block='c')
X = identity_block(X, 3, [256, 256], stage=4, block='d')
X = identity_block(X, 3, [256, 256], stage=4, block='e')
X = identity_block(X, 3, [256, 256], stage=4, block='f')

# Stage 5 (≈3 lines)
X = X = convolutional_block(X, f=3, filters=[512, 512], stage=5, block='a', s=2)
X = identity_block(X, 3, [512, 512], stage=5, block='b')
X = identity_block(X, 3, [512, 512], stage=5, block='c')

# AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

### END CODE HERE ###

# output layer
X = Flatten()(X)
X = Dense(4, activation='softmax', name='fc' + str(4))(X) # , kernel_initializer=glorot_uniform(seed=0))(X)

# Create model
model = Model(inputs=X_input, outputs=X, name='ResNet50')

In [65]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_13 (ZeroPadding2 (None, 230, 230, 3)  0           input_13[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d_13[0][0]          
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
sgd = SGD(lr=1e-2, decay=1e-4, momentum=0.9, nesterov=True)
# sgd = SGD(lr=1e-1, decay=1e-4, momentum=0.9, nesterov=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto')
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
hist = model.fit_generator(generator, steps_per_epoch=(5213*(1-VALIDATION_SPLIT)) // BATCH_SIZE, validation_data=validation_generator, \
                           validation_steps=(5213*(VALIDATION_SPLIT)) // BATCH_SIZE, epochs=25, verbose=1, callbacks=[reduce_lr])

Epoch 1/25
146/146 [==============================] - 84s 577ms/step - loss: 1.6368 - acc: 0.5020 - val_loss: 3.0577 - val_acc: 0.3893
Epoch 2/25
146/146 [==============================] - 72s 492ms/step - loss: 1.7743 - acc: 0.4795 - val_loss: 4.5057 - val_acc: 0.2541
Epoch 3/25
146/146 [==============================] - 73s 498ms/step - loss: 1.6141 - acc: 0.5155 - val_loss: 1.7227 - val_acc: 0.5492
Epoch 4/25
146/146 [==============================] - 75s 511ms/step - loss: 1.7151 - acc: 0.4274 - val_loss: 2.2698 - val_acc: 0.3361
Epoch 5/25
146/146 [==============================] - 73s 501ms/step - loss: 1.5500 - acc: 0.4151 - val_loss: 2.0259 - val_acc: 0.4180
Epoch 6/25
146/146 [==============================] - 70s 476ms/step - loss: 1.4297 - acc: 0.5174 - val_loss: 1.2173 - val_acc: 0.5225
Epoch 7/25
146/146 [==============================] - 72s 496ms/step - loss: 1.3630 - acc: 0.5348 - val_loss: 1.1109 - val_acc: 0.5615
Epoch 8/25
146/146 [==============================] - 7